In [1]:
from dotenv import load_dotenv
import os
import re

In [2]:
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Neo4jVector

#Model import
import google.generativeai as genai


d:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
load_dotenv()

True

## Embeddings setup

In [4]:
URI = 'neo4j+s://295c5d0b.databases.neo4j.io'
NEO4J_USER = os.getenv('N_USER')
NEO4J_PASSWORD = os.getenv('N_PASSWORD')
G_KEY = os.getenv('GEMINI_API')

In [5]:
# Modelo optimizado para trabajar en espanol y para hacer preguntas.
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

d:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Se nesecita para crear los embedings en el nodo . Neo4j.Vector nesecita este el model_embedding y en este caso estoy utilizando el sentence_transformer
class SentenceEmbedding:
    def __init__(self, model):
        self.model = model

    def embed_query(self, text):
        return self.model.encode(text).tolist()  # Convierte a lista

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()  # Convierte a lista

In [7]:
# Crea una instancia de SentenceEmbedding
embedding = SentenceEmbedding(model)

In [8]:
# Esto solo se ejecuta una vez para crear los embedings en el nodo utilizando el texto de la propiedad "pregunta"
Neo4j_Vector_Store = Neo4jVector.from_existing_graph(
    embedding=embedding,
    url=URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    index_name="p_vector_index",
    node_label="preguntas",
    text_node_properties=["pregunta"],
    embedding_node_property="pregunta_embedding",
)

In [9]:
# Conectarnos a la based de datos para verificar las propiedades de los nodos y confirmar de la cracion de los embeddings.
# Conexión a la base de datos Neo4j
uri = "neo4j+s://295c5d0b.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [10]:
# Hacer la consulta
def get_nodes(tx):
    query = """
    MATCH (n:preguntas) 
    RETURN n.pregunta_id AS id, n.pregunta_embedding AS embedding
    LIMIT 5    
    """
    result = tx.run(query)
    return [(record['id'], record['embedding']) for record in result]


In [11]:
with driver.session() as session:
    nodes = session.read_transaction(get_nodes)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_3964\2132304414.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  nodes = session.read_transaction(get_nodes)


In [12]:
print(nodes[:5])

[(7804, [0.1272953599691391, -0.10442045331001282, 0.13369162380695343, 0.18808569014072418, 0.17758817970752716, -0.27679452300071716, 0.03258254751563072, 0.034436628222465515, 0.10342472791671753, 0.1644621640443802, 0.35587772727012634, -0.2172115594148636, 0.04762779176235199, -0.13593770563602448, 0.15098021924495697, -0.3159516751766205, -0.40999189019203186, 0.13196533918380737, 0.06748148798942566, -0.05846607685089111, -0.0641382709145546, -0.08403285592794418, 0.05361432954668999, -0.08277302980422974, 0.07665690034627914, -0.054762694984674454, -0.1929405927658081, -0.10688961297273636, -0.23862135410308838, 0.11085798591375351, 0.09843514114618301, -0.22172389924526215, -0.07255192846059799, 0.0226422306150198, 0.1904367059469223, 0.18320168554782867, -0.14832940697669983, -0.20131425559520721, -0.14250580966472626, 0.08438070863485336, 0.0026452429592609406, 0.006097048986703157, -0.06523162871599197, -0.0801846981048584, -0.11146879196166992, -0.21393780410289764, -0.030

## Load the model

In [13]:
genai.configure(api_key = G_KEY)

In [14]:
# New Gemini model Configurations
generation_config = {
  "temperature": 0.3,
  "top_p": 0.95,
  "top_k": 5,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [15]:
llm = genai.GenerativeModel(model_name='gemini-1.5-flash-latest', generation_config=generation_config)

## Functions

In [16]:
# Función para obtener nodos relacionados pregunta-[responde]->articulo
def get_related_nodes(session, pregunta_id):
    if not pregunta_id:
        raise ValueError("Pregunta_id no puede ser None o vacío.")    
    
    # Buscar en todos los nodos preguntas
    query = """
    MATCH (start:preguntas)   
    WHERE start.pregunta_id = $pregunta_id
    CALL apoc.path.expand(
        start,              // nodo inicial
        'responde>',        // tipo de relación
        '',                 // filtro de etiquetas (vacío para aceptar todas)
        1,                  // profundidad mínima
        1                   // profundidad máxima (antes era maxLevel)
    )
    YIELD path
    RETURN last(nodes(path)).articulo_name AS articulo_name,
           last(nodes(path)).contenido AS contenido_final
    """
    
    try:
        result = session.run(query, pregunta_id=pregunta_id)
        record = next(result) if result else None
        if record:
            text = f'articulo_numero: {record["articulo_name"]}\n{record["contenido_final"]}'
            return text  # o json.dumps(diccionario)
        return ""
    except Exception as e:
        print(f"Error ejecutando la consulta: {e}")
        return ""

## Test


In [46]:
query= 'Quienes pueden votar en una asamblea general'

In [18]:
vector_store = Neo4j_Vector_Store

In [47]:
results = vector_store.similarity_search(query, k=2)

In [48]:
results

[Document(metadata={'pregunta_name': 'pregunta 2504', 'articulo_id': 25, 'pregunta_id': 2504}, page_content='\npregunta: ¿Qué sucede en casos de votación nominal en la asamblea general?'),
 Document(metadata={'pregunta_name': 'pregunta 4506', 'articulo_id': 45, 'pregunta_id': 4506}, page_content='\npregunta: ¿Cuál es la máxima mayoría que se puede exigir para una decisión en una asamblea general?')]

In [49]:
print(results[0].metadata.get('pregunta_id'),
      results[1].metadata.get('pregunta_id'))

2504 4506


In [50]:
response_1 = get_related_nodes(session=driver.session(),pregunta_id=2504)

In [51]:
print(response_1)

articulo_numero: Artículo 25
Tales coeficientes determinarán:1. La proporción de los derechos de cada uno de los propietarios de bienes privados sobre los bienes comunes del edificio o conjunto.2. El porcentaje de participación en la asamblea general de propietarios.3. El índice de participación con que cada uno de los propietarios de bienes privados ha de contribuir a las expensas comunes del edifico o conjunto, mediante el pago de cuotas ordinarias y extraordinarias de administración, salvo cuando éstas se determinen de acuerdo con los módulos de contribución en la forma señalada en el reglamento.


In [27]:
response_2 = get_related_nodes(session=driver.session(),pregunta_id=2504)

In [28]:
print(response_2)

articulo_numero: Artículo 25
Tales coeficientes determinarán:1. La proporción de los derechos de cada uno de los propietarios de bienes privados sobre los bienes comunes del edificio o conjunto.2. El porcentaje de participación en la asamblea general de propietarios.3. El índice de participación con que cada uno de los propietarios de bienes privados ha de contribuir a las expensas comunes del edifico o conjunto, mediante el pago de cuotas ordinarias y extraordinarias de administración, salvo cuando éstas se determinen de acuerdo con los módulos de contribución en la forma señalada en el reglamento.


In [130]:
response = response_1+response_2

In [131]:
response

'articulo_numero: Artículo 54\nEl consejo de administración deliberará y decidirá válidamente con la presencia y votos de la mayoría de sus miembros, salvo que el reglamento de propiedad horizontal estipule un quórum superior, con independencia de los coeficientes de copropiedad.articulo_numero: Artículo 36\nLa dirección y administración de la persona jurídica corresponde a la asamblea general de propietarios, al consejo de administración, si lo hubiere, y al administrador de edificio o conjunto.PARÁGRAFO\xa01°.\xa0Adicionado por el art. 46, Ley 2079 de 2021.\xa0<El texto adicionado es el siguiente>\xa0 Para las propiedades horizontales conformadas por 5 o menos unidades de Viviendas de Interés Social y Viviendas de Interés Prioritario, bastará con la designación de un administrador quien ejercerá sus funciones de conformidad con lo establecido en el Capítulo XI de la Ley 675 de 2001, en ausencia de esa designación, los propietarios actuarán como administradores conjuntos y serán solid

## Generar respuestas

In [54]:
my_prompt = f"Sos un asistente legal especializado en la ley 675 de Colombia y usando el siguiente contexto: {response_1} responde la siguiente pregunta:{query} de la manera mas detallada posible y si aplica utiliza algun ejemplo y al finalizar invita al usuario a volver a preguntar"

In [55]:
ans = llm.generate_content(my_prompt, generation_config=generation_config)

In [56]:
print(ans.text)

## ¿Quiénes pueden votar en una asamblea general de propietarios?

De acuerdo con el **Artículo 25 de la Ley 675 de 2001**, los propietarios de bienes privados dentro de un edificio o conjunto residencial tienen derecho a votar en la asamblea general de propietarios. 

**La participación en la asamblea se determina por el porcentaje de participación que cada propietario tiene en el edificio o conjunto, como se establece en el punto 2 del artículo 25.** Este porcentaje se calcula de acuerdo con los coeficientes que se establecen en la escritura pública de constitución del régimen de propiedad horizontal. 

**Ejemplo:**

Si un edificio tiene 10 apartamentos y cada apartamento tiene un coeficiente de 10, cada propietario tendrá un 10% de participación en la asamblea general. Esto significa que cada propietario tendrá derecho a 1 voto por cada 10% de participación.

**En resumen, los siguientes pueden votar en una asamblea general de propietarios:**

* **Propietarios de bienes privados:** 